In [7]:
import json
import xml.etree.ElementTree as et

In [8]:
class Course:
    def __init__(self, course_id, title, duration):
        self.course_id = course_id
        self.title= title
        self.duration = duration

# Normal Serializer : we will improve it latter

In [23]:
class CourseSerializer:
    def serialize(self, course, format_):
        if format_.upper() == "JSON":
            course_info = {
                "id": course.course_id,
                "title": course.title,
                "duration": course.duration
            }
            return json.dumps(course_info)
        elif format_.upper() == "XML":
            course_info = et.Element('course', attrib={'id': str(course.course_id)})
            title = et.SubElement(course_info, 'title')
            title.text = course.title
            duration = et.SubElement(course_info, 'duration')
            duration.text = course.duration
            return et.tostring(course_info, encoding='unicode')
        else:
            raise ValueError("Invalid Format")

In [24]:
course = Course(1, "Python", "3w")

In [25]:
serializer = CourseSerializer()

In [26]:
serializer.serialize(course, "json")

'{"id": 1, "title": "Python", "duration": "3w"}'

In [27]:
serializer.serialize(course, "xml")

'<course id="1"><title>Python</title><duration>3w</duration></course>'

we can see here a series of if, elif and else. To increase the readibility of the code we will create common interface

In [45]:
class CourseSerializer:
    def serialize(self, course, format_):
        serializer = self._get_serializer(format_)
        return serializer(course)
        
    def _get_serializer(self, format_):
        if format_.upper() == "JSON":
            return self._serialize_to_json
        elif format_.upper() == "XML":
            return self._serialize_to_xml
        else:
            raise ValueError("Invalid Format")
            
    def _serialize_to_json(self, course):
        course_info = {
                    "id": course.course_id,
                    "title": course.title,
                    "duration": course.duration
                }
        return json.dumps(course_info)
    
    def _serialize_to_xml(self, course):
        course_info = et.Element('course', attrib={'id': str(course.course_id)})
        title = et.SubElement(course_info, 'title')
        title.text = course.title
        duration = et.SubElement(course_info, 'duration')
        duration.text = course.duration
        return et.tostring(course_info, encoding='unicode')

In [46]:
course = Course(1, "Python", "3w")
serializer = CourseSerializer()


In [47]:
serializer.serialize(course, "json")


'{"id": 1, "title": "Python", "duration": "3w"}'

In [48]:
serializer.serialize(course, "xml")

'<course id="1"><title>Python</title><duration>3w</duration></course>'

As we can see in above code, none of the method use self inside it. So it is good to seperation those methods as independent function

In [55]:
class CourseSerializer:
    def serialize(self, course, format_):
        serializer = get_serializer(format_)
        return serializer(course)
        
def get_serializer(format_):
    if format_.upper() == "JSON":
        return serialize_to_json
    elif format_.upper() == "XML":
        return serialize_to_xml
    else:
        raise ValueError("Invalid Format")

def serialize_to_json(course):
    course_info = {
                "id": course.course_id,
                "title": course.title,
                "duration": course.duration
            }
    return json.dumps(course_info)

def serialize_to_xml(course):
    course_info = et.Element('course', attrib={'id': str(course.course_id)})
    title = et.SubElement(course_info, 'title')
    title.text = course.title
    duration = et.SubElement(course_info, 'duration')
    duration.text = course.duration
    return et.tostring(course_info, encoding='unicode')

In [56]:
course = Course(1, "Python", "3w")
serializer = CourseSerializer()


In [57]:
serializer.serialize(course, "json")

'{"id": 1, "title": "Python", "duration": "3w"}'

In [58]:
serializer.serialize(course, "xml")

'<course id="1"><title>Python</title><duration>3w</duration></course>'

# Factory Method

In [60]:
import json
from xml.etree.ElementTree import Element as et

In [61]:
class JsonSerializer:
    def __init__(self):
        self._current_object = None
    def start_object(self, object_name, object_id):
        self._current_object = {
            "id": object_id
        }
    def add_property(self, name, value):
        self._current_object[name] = value
    def to_str(self):
        return json.dumps(self._current_object)

class XmlSerializer:
    def __init__(self):
        self._element = None
    def start_object(self, object_name, object_id):
        self._element = et.Element(object_name, attrib={'id': object_id})
    def add_property(self):
        prop = et.SubElement(self._element, name)
        prop.text = value
    def to_str(self):
        return et.tostring(self._element, encoding='unicode')

In [ ]:
class ObjectSerializer:
    def serialize(self, serializable, format_):
        serializer = factory.get_serializer(format_)
        serializable.serialize(serializer)
        return serializer.to_str()